In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths to the dataset
bmask_path =  "/content/Brain_Segmentation"
gmmask_path = "/content/GM_Mask"
original_path = "/content/Original"

# Parameters
image_size = (224, 224)
batch_size = 8
epochs = 100

In [ ]:
import zipfile
import os

# Define paths for uploaded zip files
brain_segmentation_zip = "/content/Brain_Segmentation.zip"
gm_mask_zip = "/content/GM_Mask.zip"
original_zip = "/content/Original.zip"

# Function to unzip datasets
def unzip_dataset(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"{extract_to} dataset extracted.")

# Unzip Brain_Segmentation dataset
unzip_dataset(brain_segmentation_zip, "/content")

# Unzip GM_Mask dataset
unzip_dataset(gm_mask_zip, "/content")

# Unzip Original dataset
unzip_dataset(original_zip, "/content")

# Verify directory structure for Brain_Segmentation
brain_segmentation_dir = "/content/Brain_Segmentation"
gm_mask_dir = "/content/GM_Mask"
original_dir = "/content/Original"

print("Brain Segmentation Classes:", os.listdir(brain_segmentation_dir))
print("GM Mask Classes:", os.listdir(gm_mask_dir))
print("Original Classes:", os.listdir(original_dir))


In [ ]:
# Load images and masks for 3 classes: AD, MCI, CN
def load_data(image_dir, mask_dir):
    images = []
    masks = []

    for label in ["AD", "MCI", "CN"]:
        image_folder = os.path.join(image_dir, label)
        mask_folder = os.path.join(mask_dir, label)

        for image_file in os.listdir(image_folder):
            image_path = os.path.join(image_folder, image_file)
            mask_path = os.path.join(mask_folder, image_file)

            # Load image and mask in grayscale
            image = load_img(image_path, target_size=image_size, color_mode="grayscale")
            mask = load_img(mask_path, target_size=image_size, color_mode="grayscale")

            images.append(img_to_array(image) / 255.0)
            masks.append(img_to_array(mask) / 255.0)

    return np.array(images), np.array(masks)


In [ ]:
# Metrics calculation
def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-7)

def precision(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fp = np.sum((1 - y_true) * y_pred)
    return tp / (tp + fp + 1e-7)

def recall(y_true, y_pred):
    tp = np.sum(y_true * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    return tp / (tp + fn + 1e-7)

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * (p * r) / (p + r + 1e-7)

In [ ]:
# Visualization
def plot_results(images, masks, predictions, n=3):
    plt.figure(figsize=(15, n * 5))
    for i in range(n):
        # Original Image
        plt.subplot(n, 3, i * 3 + 1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        plt.title("Original Image")
        plt.axis('off')

        # Ground Truth Mask
        plt.subplot(n, 3, i * 3 + 2)
        plt.imshow(masks[i].squeeze(), cmap='gray')
        plt.title("Ground Truth Mask")
        plt.axis('off')

        # Predicted Mask
        plt.subplot(n, 3, i * 3 + 3)
        plt.imshow(predictions[i].squeeze(), cmap='gray')
        plt.title("Predicted Mask")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

def showHistory(history):
  # list all data in history
  print(history.history.keys())
  fig = plt.figure(figsize=(10,6))

  # summarize history for accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy',fontsize=20)
  plt.ylabel('Accuracy',fontsize=20)
  plt.xlabel('Epoch',fontsize=20)
  plt.legend(['train', 'test'], loc='center right',fontsize=20)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.tick_params(axis='both', which='minor', labelsize=18)
  plt.show()

  # summarize history for loss
  fig = plt.figure(figsize=(10,6))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss',fontsize=20)
  plt.ylabel('Loss',fontsize=20)
  plt.xlabel('Epoch',fontsize=20)
  plt.legend(['train', 'test'], loc='center right',fontsize=20)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.tick_params(axis='both', which='minor', labelsize=18)
  plt.show()

In [ ]:
import tensorflow.keras.backend as K

# Custom Loss Function to handle imbalanced classes
def balanced_cross_entropy(y_true, y_pred, beta=0.5):
    """
    Computes the balanced cross-entropy loss.

    Args:
        y_true: Ground truth labels, shape = (batch_size, height, width, 1).
        y_pred: Predicted labels, shape = (batch_size, height, width, 1).
        beta: Balancing factor, should be between 0 and 1.

    Returns:
        Loss value.
    """
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    # Compute class weights
    pos_weight = beta / (1 - beta)

    # Balanced cross-entropy
    loss = -beta * y_true * K.log(y_pred + K.epsilon()) - (1 - beta) * (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    return K.mean(loss)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
)

# Dice Loss
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

# Combined Dice + Balanced Cross-Entropy Loss
def dice_bce_loss(y_true, y_pred):
    return balanced_cross_entropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:
# Modified U-Net architecture
def Multi-Layer-U-Net(input_size):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.3)(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Additional encoder layer
    c5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    p5 = MaxPooling2D((2, 2))(c5)

    # Bottleneck
    c6 = Conv2D(2048, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p5)
    c6 = Dropout(0.4)(c6)
    c6 = Conv2D(2048, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    # Decoder
    u7 = Conv2DTranspose(1024, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c5])
    c7 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.3)(c7)
    c7 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c4])
    c8 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.3)(c8)
    c8 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c3])
    c9 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    u10 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c9)
    u10 = concatenate([u10, c2])
    c10 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u10)
    c10 = Dropout(0.1)(c10)
    c10 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)

    u11 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c10)
    u11 = concatenate([u11, c1])
    c11 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u11)
    c11 = Dropout(0.1)(c11)
    c11 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c11)

    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c11)

    model = Model(inputs, outputs)
    return model

**GM Mask Segmentation Multi-Layer UNet**

In [ ]:
# Load dataset
images, masks = load_data(original_path, gmmask_path)

# Split into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Compile the model
model = Multi-Layer-U-Net((224, 224, 1))
model.compile(optimizer=Adam(learning_rate=1e-4), loss=balanced_cross_entropy, metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=batch_size,
    epochs=epochs
)

model.save('GrayMatter_Segmentation-UNet.keras')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 224, 224, 64)   │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 224, 224, 64)   │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 224, 224, 64)   │         36,928 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 112, 112, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 112, 112, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 112, 112, 128)  │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 112, 112, 128)  │        147,584 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 56, 56, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 56, 56, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 56, 56, 256)    │              0 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 56, 56, 256)    │        590,080 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 28, 28, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 28, 28, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 28, 28, 512)    │              0 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 28, 28, 512)    │      2,359,808 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 14, 14, 512)    │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)    

 Total params: 124,361,025 (474.40 MB)

 Trainable params: 124,361,025 (474.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.8720 - loss: 0.2805 - val_accuracy: 0.8826 - val_loss: 0.1766
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 72s 549ms/step - accuracy: 0.8835 - loss: 0.1285 - val_accuracy: 0.8826 - val_loss: 0.1460
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 566ms/step - accuracy: 0.8852 - loss: 0.1136 - val_accuracy: 0.8683 - val_loss: 0.1353
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 41s 566ms/step - accuracy: 0.9097 - loss: 0.0933 - val_accuracy: 0.8766 - val_loss: 0.1227
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 41s 563ms/step - accuracy: 0.9175 - loss: 0.0862 - val_accuracy: 0.8936 - val_loss: 0.1025
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 570ms/step - accuracy: 0.9230 - loss: 0.0813 - val_accuracy: 0.8944 - val_loss: 0.1016
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 563ms/step - accuracy: 0.9332 - loss: 0.0722 - val_accuracy: 0.9168 - val_loss: 0.0822
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 576ms/step - accuracy: 0.9278 - loss: 0.0764 - va

In [ ]:
# Evaluate the model
evaluation = model.evaluate(x_test, y_test)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}\n\n")

showHistory(history)

In [ ]:
# Predict on test data
predictions = model.predict(x_test)
predictions = (predictions > 0.5).astype(np.float32)

# Compute metrics
metrics = {
    "Accuracy": evaluation[0],
    "Loss": evaluation[1],
    "Dice Coefficient": dice_coefficient(y_test, predictions),
    "Precision": precision(y_test, predictions),
    "Recall": recall(y_test, predictions),
    "F1 Score": f1_score(y_test, predictions)
}

# Display metrics in a DataFrame
metrics_df = pd.DataFrame(metrics, index=["Metrics"])
metrics_df.head()